In [11]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA

# Load data from CSVs
sales_data = pd.read_csv("sales_data.csv", sep=';')
store_master = pd.read_csv("store_master.csv", sep=';')

# Data Preprocessing
sales_data['unit_price'] = sales_data['unit_price'].str.replace(',', '.').astype(float)
sales_data['date'] = pd.to_datetime(sales_data['date'])
store_master['latitude'] = store_master['latitude'].str.replace(',', '.').astype(float)
store_master['longitude'] = store_master['longitude'].str.replace(',', '.').astype(float)

# Merging data
data = pd.merge(sales_data, store_master, on='store')

# Calculate revenue
data['revenue'] = data['unit_price'] * data['qty']

#  Before Data Analysis - Documentation
print("""
Before proceeding with data analysis, I decided to include all available data as it provides comprehensive insights into sales trends.
I used variables such as store, unit price, and item category for analysis, as they are crucial in understanding sales dynamics.
""")

#  Choose 10 Items to Forecast - Justification
print("""
I chose the top 10 items by quantity sold as they represent the highest demand in the dataset.
Forecasting these items will provide valuable insights into future sales trends.
""")

top_items = data.groupby('item')['qty'].sum().nlargest(10).index.tolist()
top_data = data[data['item'].isin(top_items)]

#  Model Data
# Multivariate Regression Model
X = top_data[['store', 'unit_price', 'item_category']]
y = top_data['qty']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
regression_model = LinearRegression()
regression_model.fit(X_train, y_train)

#  Evaluate the Regression Model
regression_train_preds = regression_model.predict(X_train)
regression_test_preds = regression_model.predict(X_test)
regression_train_rmse = mean_squared_error(y_train, regression_train_preds, squared=False)
regression_test_rmse = mean_squared_error(y_test, regression_test_preds, squared=False)

#  Model Data - Time Series Forecasting Model (ARIMA)
forecast_results = {}
for item in top_items:
    item_data = top_data[top_data['item'] == item].copy()
    item_data['date'] = pd.to_datetime(item_data['date'])

    # Remove duplicate dates or aggregate data for those dates
    item_data = item_data.groupby('date').sum()

    item_data = item_data.sort_index()
    item_data = item_data.asfreq('D')

    model = ARIMA(item_data['qty'], order=(5,1,0))
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=30)
    forecast_results[item] = forecast

#  Estimated Future Values
future_input_variables = {
    'store': X_train.iloc[-1]['store'],
    'unit_price': X_train.iloc[-1]['unit_price'] * 1.05,  # Increase by 5%
    'item_category': X_train.iloc[-1]['item_category']
}

#  Error Rate and Self-feedback
print(f"""
Error Measure: Root Mean Squared Error (RMSE)
The Regression Model RMSE (Train): {regression_train_rmse}
The Regression Model RMSE (Test): {regression_test_rmse}

The ARIMA Model RMSE:
""")
arima_rmse = {}
for item, forecast in forecast_results.items():
    actual_values = top_data[top_data['item'] == item]['qty'].values[-len(forecast):]
    rmse = mean_squared_error(actual_values, forecast, squared=False)
    arima_rmse[item] = rmse
    print(f"Item {item}: {rmse}")

#  Results and Output
print(f"""
Discussion:
I set up the analysis by first loading and preprocessing the data.
I then chose to include all available variables for analysis to provide comprehensive insights into sales trends.
I selected the top 10 items by quantity sold for forecasting as they represent the highest demand in the dataset.
I implemented a multivariate regression model and an ARIMA model for forecasting.
The regression model performed with an RMSE of {regression_train_rmse} on the training data and {regression_test_rmse} on the test data.
The ARIMA model achieved RMSE values as printed above.

Numerical Results:
Forecasted Quantity for the Next 30 Days:
""")
for item, forecast in forecast_results.items():
    print(f"Item {item}: {forecast.tolist()}")

# Results and Output - Graphs
print("""
Graphs:
1. Bar chart: Total Revenue per Store
2. Bar chart: Total Revenue per Item
3. Bar chart: Total Revenue per Item Category
4. Scatter plot: Unit Price vs Quantity Sold
5. Line chart: Date vs Total Revenue
6. Line chart: Date vs Average Unit Price
7. Line chart: Date vs Total Quantity Sold
8. Line chart: Forecasted Quantity for the Next 30 Days
""")
# Calculate total revenue per store
store_revenue = data.groupby('store')['revenue'].sum().reset_index()

# Calculate total revenue per item
item_revenue = data.groupby('item')['revenue'].sum().reset_index()

# Calculate total revenue per item category
item_category_revenue = data.groupby('item_category')['revenue'].sum().reset_index()

# Calculate total revenue for each date
revenue_by_date = data.groupby('date')['revenue'].sum().reset_index()

# Calculate average unit price for each date
avg_unit_price_by_date = data.groupby('date')['unit_price'].mean().reset_index()

# Calculate total quantity sold for each date
qty_by_date = data.groupby('date')['qty'].sum().reset_index()

# Plotting total revenue per store
fig1 = px.bar(store_revenue, x='store', y='revenue', title='Total Revenue per Store', labels={'store': 'Store', 'revenue': 'Total Revenue'})
fig1.show()

# Plotting total revenue per item
fig2 = px.bar(item_revenue, x='item', y='revenue', title='Total Revenue per Item', labels={'item': 'Item', 'revenue': 'Total Revenue'})
fig2.show()

# Plotting total revenue per item category
fig3 = px.bar(item_category_revenue, x='item_category', y='revenue', title='Total Revenue per Item Category', labels={'item_category': 'Item Category', 'revenue': 'Total Revenue'})
fig3.show()

# Plotting unit price vs quantity sold
fig4 = px.scatter(data, x='unit_price', y='qty', title='Unit Price vs Quantity Sold', labels={'unit_price': 'Unit Price', 'qty': 'Quantity Sold'})
fig4.show()

# Plotting date vs total revenue
fig5 = px.line(revenue_by_date, x='date', y='revenue', title='Date vs Total Revenue', labels={'date': 'Date', 'revenue': 'Total Revenue'})
fig5.show()

# Plotting date vs average unit price
fig6 = px.line

# Plotting forecasted quantity for the next 30 days for each item
for item, forecast in forecast_results.items():
    dates = pd.date_range(start=data['date'].max() + pd.Timedelta(days=1), periods=len(forecast))
    fig = px.line(x=dates, y=forecast, title=f'Forecasted Quantity for Item: {item}', labels={'x': 'Date', 'y': 'Quantity'})
    fig.show()


Output hidden; open in https://colab.research.google.com to view.